# Spotify Podcast Dataset - Publisher



In [51]:
import pandas as pd
import altair as alt
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
import numpy as np

In [52]:
df = pd.read_csv('../../podcasts-no-audio-13GB-selected/metadata_with_episode_dates.csv')
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d').reset_index(drop=True)

# Dataset is constrained to start_date and end_date
start_date = pd.to_datetime("2019-01-01")
end_date = pd.to_datetime("2020-03-01")

# Filter the DataFrame to include only rows between start_date and end_date
df = df[(df.release_date >= start_date) & (df.release_date <= end_date)]

In [53]:
df.shape

(87352, 13)

In [54]:
df.columns

Index(['show_uri', 'show_name', 'show_description', 'publisher', 'language',
       'rss_link', 'episode_uri', 'episode_name', 'episode_description',
       'duration', 'show_filename_prefix', 'episode_filename_prefix',
       'release_date'],
      dtype='object')

# Publisher Specific Insights

In [55]:
df['publisher'].unique()

array(['Katie Houle', 'Morning Cup Of Murder', 'Inside the 18 GK Media',
       ..., 'High Schooler ', 'Zach Valen', 'Chelsea Batalla'],
      dtype=object)

## Publisher Summary Stats

In [56]:
df_ss = df[df['publisher']=='Spotify Studios']

In [57]:
df_ss.shape

(657, 13)

In [58]:
df_pub_barchart_values = np.array([
    # number of shows
    df_ss['show_filename_prefix'].unique().shape[0],
    # number of total episodes
    df_ss['episode_filename_prefix'].unique().shape[0],
    # average number of episodes per show
    df_ss.groupby('show_filename_prefix').size().mean()
])
df_pub_barchart_values

array([ 35.        , 657.        ,  18.77142857])

In [59]:
df_pub_barchart_categories = np.array(['number_of_shows', 'number_of_episodes', 'mean_number_of_episodes_per_show'])

In [60]:
cols = ['Category','Values']
df_pub_barchart = pd.DataFrame(np.array([df_pub_barchart_categories, df_pub_barchart_values]).T, columns=cols)
df_pub_barchart

,Category,Values
0,number_of_shows,35.0
1,number_of_episodes,657.0
2,mean_number_of_episodes_per_show,18.771428571428572


In [61]:
df_pub_barchart.dtypes

Category    object
Values      object
dtype: object

In [62]:
# bar chart of number of shows, number of episodes, number of episodes per show
bar_chart = alt.Chart(df_pub_barchart).mark_bar().encode(
    x='Values:Q',  # Quantitative scale for the y-axis
    y=alt.X('Category:N'),
    tooltip=['Values']
).configure_scale(
    bandPaddingInner=0.2,
)

bar_chart = bar_chart.properties(width=300, height=200)
bar_chart.save('publisher_summary_bar_chart.json')
bar_chart

alt.Chart(...)

## Publisher Timeseries Stats

In [63]:
df.columns

Index(['show_uri', 'show_name', 'show_description', 'publisher', 'language',
       'rss_link', 'episode_uri', 'episode_name', 'episode_description',
       'duration', 'show_filename_prefix', 'episode_filename_prefix',
       'release_date'],
      dtype='object')

In [64]:
df.dtypes

show_uri                           object
show_name                          object
show_description                   object
publisher                          object
language                           object
rss_link                           object
episode_uri                        object
episode_name                       object
episode_description                object
duration                          float64
show_filename_prefix               object
episode_filename_prefix            object
release_date               datetime64[ns]
dtype: object

In [65]:
df['release_date']

0        2019-12-18
1        2020-01-30
2        2019-01-18
3        2019-03-01
5        2019-10-16
            ...    
105355   2019-03-29
105356   2019-05-04
105357   2019-01-08
105358   2019-06-25
105359   2019-10-27
Name: release_date, Length: 87352, dtype: datetime64[ns]

In [66]:
df_ss = df[df['publisher']=='Spotify Studios']

In [82]:

# First group by show name AND aggregate time by month AND THEN get the counts which will give you the episodes per month by show for every month.
# then take the average over all months to get the average episodes per month by show. 
monthly_counts = df_ss[['show_name','release_date']].groupby(['show_name',df_ss.release_date.dt.to_period("M")]).agg('count') \    
    .groupby('show_name').mean().sort_values(by='release_date',ascending=False)
monthly_counts = monthly_counts.reset_index()

# Reset the index to have 'Date' as a column again
monthly_counts['episodes_per_month'] = monthly_counts['release_date']
monthly_counts.drop('release_date',axis=1,inplace=True)
# Display the resulting DataFrame
monthly_counts.head()


ValueError: cannot insert release_date, already exists

In [79]:
monthly_counts.head()

,show_name,episodes_per_month
0,The Hottest Take,15.000000
1,Spotify x IPG Mediabrands,12.000000
2,21 Days with mxmtoon,8.000000
3,Jemele Hill is Unbothered,7.545455
4,Football Legends,6.200000


In [70]:
# bar chart of number of shows, number of episodes, number of episodes per show
bar_chart2 = alt.Chart(monthly_counts).mark_bar().encode(
    x='episodes_per_month:Q',  # Quantitative scale for the y-axis
    y=alt.Y('show_name:N')
).configure_scale(
    bandPaddingInner=0.2,
)

bar_chart2 = bar_chart2.properties(width=300, height=500)
bar_chart2


alt.Chart(...)

In [80]:
monthly_counts.value_counts().head()

show_name                                     episodes_per_month
21 Days with mxmtoon                          8.000000              1
The Hottest Take                              15.000000             1
Noisemakers                                   2.333333              1
Riggle's Picks with Rob Riggle & Sarah Tiana  4.071429              1
Set It Straight with Midland                  3.666667              1
Name: count, dtype: int64

# General Insights for Publishers

## Find Shows by Publisher

In [72]:
df_pub2show = df.groupby(['publisher','show_name']).size().sort_values(ascending=False).reset_index().groupby('publisher').size().sort_values(ascending=False)
df_pub2show = pd.DataFrame(df_pub2show, columns=['Number of Shows']).reset_index()
# Calculate circle sizes based on 'Value'
#df_pub2show['CircleSize'] = np.sqrt(df_pub2show['Number of Shows'])
df_pub2show['Number of Shows Jitter'] = df_pub2show['Number of Shows'] + np.abs(np.random.normal(0,1,(len(df_pub2show),)))

## Find Episodes by Publisher

In [73]:
df_pub2episode = df.groupby('publisher').size().sort_values(ascending=False)
df_pub2episode = pd.DataFrame(df_pub2episode, columns=['Number of Episodes']).reset_index()
# Calculate circle sizes based on 'Value'
df_pub2episode['CircleSize'] = np.sqrt(df_pub2episode['Number of Episodes'])
df_pub2episode['Number of Episodes Jitter'] = df_pub2episode['Number of Episodes'] + np.random.random((len(df_pub2episode),))
df_pub2episode.shape

(14983, 4)

In [74]:
df_pub = df_pub2episode.merge(df_pub2show,on='publisher',how='left')

In [75]:
import altair as alt
import pandas as pd
import numpy as np

# Create a scatter plot with circles to represent the categorical table
circle_table = alt.Chart(df_pub.sample(5000)).mark_circle().encode(
    x=alt.X('publisher:O', axis=alt.Axis(labels=False, title='')),
    y=alt.Y('Number of Episodes Jitter:Q', scale=alt.Scale(type='log'), axis=alt.Axis(labels=False, title='')),
    size=alt.Size('CircleSize:Q', scale=alt.Scale(range=[5, 2000]), legend=None),  # Adjust size range as needed
    #color=alt.Color('publisher:N', legend=None),
    color=alt.Color(scale=alt.Scale(scheme='blueorange-8')),
    tooltip=['publisher:N', 'Number of Episodes:Q','Number of Shows:Q']  # Add a tooltip with 'Category' and 'Value'
).properties(
    width=700,
    height=700,
    title='Podcast Publishers (Hover Over)'
).configure_title(
    fontSize=16,
    anchor='middle'
).configure_axis(
    grid=False  # Remove gridlines
).configure_axisY(
    ticks=False
)

circle_table

alt.Chart(...)

In [76]:
df.columns

Index(['show_uri', 'show_name', 'show_description', 'publisher', 'language',
       'rss_link', 'episode_uri', 'episode_name', 'episode_description',
       'duration', 'show_filename_prefix', 'episode_filename_prefix',
       'release_date'],
      dtype='object')

In [77]:
min(np.random.normal(0,1,(len(df_pub2show),)))

-3.8083282858671543

In [78]:
df[df['publisher']=='David'].groupby('show_filename_prefix').size()

show_filename_prefix
show_0tdzFF8Qse4UR8pn9ReNL1    1
show_11UVF4OT5CAlvb5sZCsXu0    1
show_1avz49Hd2W7nj2XXb0SQu1    1
show_2b9g1RKvE30BKJDkkui16u    5
show_7ndWuoDaVWUbdMdtGCr6ab    1
dtype: int64